In [52]:
import medmnist
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from monai.data import DataLoader
import torchvision.transforms as transforms

import medmnist
from medmnist import INFO, Evaluator

from monai.networks.nets import resnet18, resnet50

In [47]:
download = True
NUM_EPOCHS = 10
BATCH_SIZE = 128
lr = 0.001
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

In [54]:
# Data loading and preprocessing
data_flag = 'organmnist3d'  # example dataset, you can change this to other datasets
info = INFO[data_flag]
task = info['task']
n_classes = len(info['label'])
DataClass = getattr(medmnist, info['python_class'])

# preprocessing
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# load the data
train_dataset = DataClass(split='train', download=True)
val_dataset = DataClass(split='val', download=download)
test_dataset = DataClass(split='test', download=download)

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)


Using downloaded and verified file: /home/ikboljonsobirov/.medmnist/organmnist3d.npz
Using downloaded and verified file: /home/ikboljonsobirov/.medmnist/organmnist3d.npz
Using downloaded and verified file: /home/ikboljonsobirov/.medmnist/organmnist3d.npz


In [59]:
for x, y in train_loader:
    print(x.shape, y.shape, x.dtype)
    break


torch.Size([2, 1, 28, 28, 28]) torch.Size([2, 1]) torch.float64


In [56]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import medmnist
from medmnist import INFO, Evaluator

# Model Definition
model = resnet18(spatial_dims=3, n_input_channels=1, num_classes=n_classes)  # Adjust parameters as needed
model.to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
for epoch in range(NUM_EPOCHS):  # Number of epochs
    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data.to(device))
        target = torch.argmax(target.to(device), dim=1)
        # print(output.shape, target.shape)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            output = model(data.to(device))
            target = torch.argmax(target.to(device), dim=1)
            val_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    val_loss /= len(val_loader.dataset)
    print(f'Epoch: {epoch}, Validation set: Average loss: {val_loss:.4f}, Accuracy: {correct}/{len(val_loader.dataset)} ({100. * correct / len(val_loader.dataset):.0f}%)')

# Testing
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data.to(device))
        target = torch.argmax(target.to(device), dim=1)
        test_loss += criterion(output, target).item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)
print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)')


RuntimeError: Input type (torch.cuda.DoubleTensor) and weight type (torch.cuda.FloatTensor) should be the same

In [72]:
import medmnist
from medmnist import INFO, Evaluator
import torch
import monai
from monai.networks.nets import DenseNet121
from monai.transforms import Compose, ScaleIntensityd, ToTensord
from monai.metrics import ROCAUCMetric #, AccuracyMetric, F1ScoreMetric, PrecisionMetric, RecallMetric
from monai.data import DataLoader
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
import torchvision.transforms as transforms
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support, accuracy_score

# Load organmnist3d dataset
info = INFO['organmnist3d']
n_channels = info['n_channels']
n_classes = len(info['label'])
DataClass = getattr(medmnist, info['python_class'])

# preprocessing transformations
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])


# datasets
train_dataset = DataClass(split='train', download=True)
val_dataset = DataClass(split='val', download=True)

# data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

# Define the model, loss function, optimizer
model = DenseNet121(
    spatial_dims=3,
    in_channels=n_channels,
    out_channels=n_classes
).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

loss_function = CrossEntropyLoss()
optimizer = Adam(model.parameters(), 1e-4)

# Define metrics
metrics = {
    'roc_auc': [],
    'accuracy': [],
    'f1_score': [],
    'precision': [],
    'recall': []
}

# Training and Validation Loop
def train_epoch(model, data_loader, loss_function, optimizer, device):
    model.train()
    epoch_loss = 0
    for batch_data in data_loader:
        inputs, labels = batch_data[0].to(device), batch_data[1].to(device)
        print(inputs.shape, labels.shape, inputs.dtype, labels.dtype)
        inputs = inputs.type(torch.DoubleTensor)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

def validate(model, data_loader, device, metrics):
    model.eval()
    with torch.no_grad():
        for metric in metrics.values():
            metric.reset()
        for batch_data in data_loader:
            inputs, labels = batch_data[0].to(device), batch_data[1].to(device)
            outputs = model(inputs)
            precision, recall, f1, _ = precision_recall_fscore_support(outputs, labels, average='macro')
    return precision, recall, f1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, loss_function, optimizer, device)
    val_metrics = validate(model, val_loader, device, metrics)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Validation Metrics: {val_metrics}")

# MC Dropout Implementation
def mc_dropout_predict(model, data_loader, num_samples, device):
    model.train()  # Enable dropout
    predictions = []
    with torch.no_grad():
        for _ in range(num_samples):
            batch_predictions = [model(batch_data[0].to(device)) for batch_data in data_loader]
            predictions.append(torch.stack(batch_predictions))
    return torch.mean(torch.stack(predictions), dim=0)  # Averaging predictions

# Use MC Dropout for uncertainty estimation (after training)
mc_predictions = mc_dropout_predict(model, val_loader, 10, device)
# Analyze mc_predictions as needed


Using downloaded and verified file: /home/ikboljonsobirov/.medmnist/organmnist3d.npz
Using downloaded and verified file: /home/ikboljonsobirov/.medmnist/organmnist3d.npz
torch.Size([64, 1, 28, 28, 28]) torch.Size([64, 1]) torch.float64 torch.int64


NotImplementedError: Could not run 'aten::slow_conv3d_forward' with arguments from the 'CUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::slow_conv3d_forward' is only available for these backends: [CPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMTIA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradMeta, AutogradNestedTensor, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDi